In [1]:
import sys
sys.path.append('..')
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from sympy import simplify_logic
import time
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.tree import _tree, export_text

from deep_logic.utils.base import validate_network, set_seed, tree_to_formula
from deep_logic.utils.relunn import get_reduced_model, prune_features
from deep_logic.utils.sigmoidnn import prune_equal_fanin
from deep_logic import logic

results_dir = 'results/xor'
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
concepts = ['c1', 'c2', 'c3', 'c4']
n_rep = 10

In [2]:
# XOR problem
x_train = torch.tensor([
    [0, 0, 0, 1],
    [0, 1, 0, 1],
    [1, 0, 0, 1],
    [1, 1, 0, 1],
], dtype=torch.float)
y_train = torch.tensor([0, 1, 1, 0], dtype=torch.long)


x_test = torch.tensor([
    [0, 0.95, 0, 1],
    [0, 0.9, 0.2, 0.7],
    [0.05, 1, 0.5, 1],
    [0.1, 0.8, 0.1, 0.8],
    [0.45, 1, 0.95, 0.9],
    [0, 0.35, 0.95, 0.9],
    [0.95, 0.9, 0.75, 0.2],
    [0.75, 0.2, 0.75, 0.15],
    [0.75, 0.15, 0.75, 0.2],
], dtype=torch.float)
y_test = torch.tensor([1, 1, 1, 1, 1, 0, 0, 1, 1], dtype=torch.long)

In [3]:
def train_nn(x_train, y_train, need_pruning, seed, device):
    set_seed(seed)
    x_train = x_train.to(device)
    y_train = y_train.to(device)
    layers = [
        torch.nn.Linear(x_train.size(1), 10),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(10, 4),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(4, 2),
        torch.nn.Softmax(dim=1),
    ]
    model = torch.nn.Sequential(*layers).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_form = torch.nn.CrossEntropyLoss()
    model.train()
    for epoch in range(1001):
        # forward pass
        optimizer.zero_grad()
        y_pred = model(x_train)
        # Compute Loss
        loss = loss_form(y_pred, y_train)

        for module in model.children():
            if isinstance(module, torch.nn.Linear):
                loss += 0.001 * torch.norm(module.weight, 1)

        # backward pass
        loss.backward()
        optimizer.step()

        if epoch > 500 and need_pruning:
            prune_features(model, n_classes=2, device=device)
            need_pruning = False
            
        # compute accuracy
        if epoch % 500 == 0:
            y_pred_d = torch.argmax(y_pred, dim=1)
            accuracy = y_pred_d.eq(y_train).sum().item() / y_train.size(0)
            print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
    
    return model

# General pruning

In [4]:
need_pruning = True
method = 'pruning'
methods = []
splits = []
explanations = []
explanations_inv = []
model_accuracies = []
explanation_accuracies = []
explanation_accuracies_inv = []
elapsed_times = []
elapsed_times_inv = []
for seed in range(n_rep):
    print(f'Seed [{seed+1}/{n_rep}]')
    
    model = train_nn(x_train, y_train, need_pruning, seed, device)
    
    y_preds = model(x_test.to(device)).cpu().detach().numpy()
    model_accuracy = accuracy_score(y_test.cpu().detach().numpy(), y_preds.argmax(axis=1))
    print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
    
    # positive class
    target_class = 1
    start = time.time()
    global_explanation, _, _ = logic.relunn.combine_local_explanations(model, 
                                                                       x_train.to(device), y_train.to(device), 
                                                                       target_class=target_class,
                                                                       method=method, device=device)
    elapsed_time = time.time() - start
    if global_explanation:
        explanation_accuracy, _ = logic.base.test_explanation(global_explanation, target_class, x_test, y_test)
        explanation = logic.base.replace_names(global_explanation, concepts)
    print(f'\t Class {target_class} - Global explanation: "{explanation}" - Accuracy: {explanation_accuracy:.4f}')
    print(f'\t Elapsed time {elapsed_time}')
        
    # negative class
    target_class = 0
    start = time.time()
    global_explanation_inv, _, _ = logic.relunn.combine_local_explanations(model, 
                                                                           x_train.to(device), y_train.to(device), 
                                                                           target_class=target_class,
                                                                           method=method, device=device)
    elapsed_time_inv = time.time() - start
    if global_explanation_inv:
        explanation_accuracy_inv, _ = logic.base.test_explanation(global_explanation_inv, target_class, x_test, y_test)
        explanation_inv = logic.base.replace_names(global_explanation_inv, concepts)
    print(f'\t Class {target_class} - Global explanation: "{explanation_inv}" - Accuracy: {explanation_accuracy_inv:.4f}')
    print(f'\t Elapsed time {elapsed_time_inv}')
    
    methods.append(method)
    splits.append(seed)
    explanations.append(explanation)
    explanations_inv.append(explanation_inv)
    model_accuracies.append(model_accuracy)
    explanation_accuracies.append(explanation_accuracy)
    explanation_accuracies_inv.append(explanation_accuracy_inv)
    elapsed_times.append(elapsed_time)
    elapsed_times_inv.append(elapsed_time_inv)

Seed [1/10]
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 1.0000
	 Epoch 1000: train accuracy: 1.0000
	 Model's accuracy: 1.0000
	 Class 1 - Global explanation: "(c1 & ~c2) | (c2 & ~c1)" - Accuracy: 1.0000
	 Elapsed time 0.023936748504638672
	 Class 0 - Global explanation: "(c1 & c2) | (~c1 & ~c2)" - Accuracy: 1.0000
	 Elapsed time 0.024935483932495117
Seed [2/10]
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 1.0000
	 Epoch 1000: train accuracy: 1.0000
	 Model's accuracy: 1.0000
	 Class 1 - Global explanation: "(c1 & ~c2) | (c2 & ~c1)" - Accuracy: 1.0000
	 Elapsed time 0.01894378662109375
	 Class 0 - Global explanation: "(c1 & c2) | (~c1 & ~c2)" - Accuracy: 1.0000
	 Elapsed time 0.01795220375061035
Seed [3/10]
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 1.0000
	 Epoch 1000: train accuracy: 1.0000
	 Model's accuracy: 0.8889
	 Class 1 - Global explanation: "(c1 & ~c2) | (c2 & ~c1)" - Accuracy: 1.0000
	 Elapsed time 0.01892995834350586


In [5]:
results_pruning = pd.DataFrame({
    'method': methods,
    'split': splits,
    'explanation': explanations,
    'explanation_inv': explanations_inv,
    'model_accuracy': model_accuracies,
    'explanation_accuracy': explanation_accuracies,
    'explanation_accuracy_inv': explanation_accuracies_inv,
    'elapsed_time': elapsed_times,
    'elapsed_time_inv': elapsed_times_inv,
})
results_pruning.to_csv(os.path.join(results_dir, 'results_pruning.csv'))
results_pruning

,method,split,explanation,explanation_inv,model_accuracy,explanation_accuracy,explanation_accuracy_inv,elapsed_time,elapsed_time_inv
0,pruning,0,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.000000,1.000000,0.023937,0.024935
1,pruning,1,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.000000,1.000000,0.018944,0.017952
2,pruning,2,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),0.888889,1.000000,1.000000,0.018930,0.016954
3,pruning,3,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.000000,1.000000,0.019947,0.015957
4,pruning,4,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.000000,1.000000,0.016955,0.015987
5,pruning,5,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.000000,1.000000,0.034869,0.018954
6,pruning,6,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.000000,1.000000,0.019948,0.017951
7,pruning,7,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.000000,1.000000,0.017959,0.019938
8,pruning,8,c1 | ~c1,(c1 & c2) | (~c1 & ~c2),1.000000,0.777778,1.000000,0.013962,0.017958
9,pruning,9,c1 | c2,~c1 & ~c2,0.888889,1.000000,0.888889,0.018936,0.013962


# LIME

In [6]:
need_pruning = False
method = 'lime'
methods = []
splits = []
explanations = []
explanations_inv = []
model_accuracies = []
explanation_accuracies = []
explanation_accuracies_inv = []
elapsed_times = []
elapsed_times_inv = []
for seed in range(n_rep):
    print(f'Seed [{seed+1}/{n_rep}]')
    
    model = train_nn(x_train, y_train, need_pruning, seed, device)
    
    y_preds = model(x_test.to(device)).cpu().detach().numpy()
    model_accuracy = accuracy_score(y_test.cpu().detach().numpy(), y_preds.argmax(axis=1))
    print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
    
    # positive class
    target_class = 1
    start = time.time()
    global_explanation, _, _ = logic.relunn.combine_local_explanations(model, 
                                                                       x_train.to(device), y_train.to(device), 
                                                                       target_class=target_class,
                                                                       method=method, device=device)
    elapsed_time = time.time() - start
    if global_explanation:
        explanation_accuracy, _ = logic.base.test_explanation(global_explanation, target_class, x_test, y_test)
        explanation = logic.base.replace_names(global_explanation, concepts)
    print(f'\t Class {target_class} - Global explanation: "{explanation}" - Accuracy: {explanation_accuracy:.4f}')
    print(f'\t Elapsed time {elapsed_time}')
        
    # negative class
    target_class = 0
    start = time.time()
    global_explanation_inv, _, _ = logic.relunn.combine_local_explanations(model, 
                                                                           x_train.to(device), y_train.to(device), 
                                                                           target_class=target_class,
                                                                           method=method, device=device)
    elapsed_time_inv = time.time() - start
    if global_explanation_inv:
        explanation_accuracy_inv, _ = logic.base.test_explanation(global_explanation_inv, target_class, x_test, y_test)
        explanation_inv = logic.base.replace_names(global_explanation_inv, concepts)
    print(f'\t Class {target_class} - Global explanation: "{explanation_inv}" - Accuracy: {explanation_accuracy_inv:.4f}')
    print(f'\t Elapsed time {elapsed_time_inv}')
    
    methods.append(method)
    splits.append(seed)
    explanations.append(explanation)
    explanations_inv.append(explanation_inv)
    model_accuracies.append(model_accuracy)
    explanation_accuracies.append(explanation_accuracy)
    explanation_accuracies_inv.append(explanation_accuracy_inv)
    elapsed_times.append(elapsed_time)
    elapsed_times_inv.append(elapsed_time_inv)

Seed [1/10]
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 1.0000
	 Epoch 1000: train accuracy: 1.0000
	 Model's accuracy: 1.0000
	 Class 1 - Global explanation: "(c1 & ~c2) | (c2 & ~c1)" - Accuracy: 1.0000
	 Elapsed time 0.8248798847198486
	 Class 0 - Global explanation: "(c1 & c2) | (~c1 & ~c2)" - Accuracy: 1.0000
	 Elapsed time 0.7699460983276367
Seed [2/10]
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 1.0000
	 Epoch 1000: train accuracy: 1.0000
	 Model's accuracy: 1.0000
	 Class 1 - Global explanation: "(c1 & ~c2) | (c2 & ~c1)" - Accuracy: 1.0000
	 Elapsed time 0.831770658493042
	 Class 0 - Global explanation: "(c1 & c2) | (~c1 & ~c2)" - Accuracy: 1.0000
	 Elapsed time 0.8936104774475098
Seed [3/10]
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 1.0000
	 Epoch 1000: train accuracy: 1.0000
	 Model's accuracy: 0.8889
	 Class 1 - Global explanation: "(c1 & ~c2) | (c2 & ~c1)" - Accuracy: 1.0000
	 Elapsed time 0.7918827533721924
	 Class 

In [8]:
results_lime = pd.DataFrame({
    'method': methods,
    'split': splits,
    'explanation': explanations,
    'explanation_inv': explanations_inv,
    'model_accuracy': model_accuracies,
    'explanation_accuracy': explanation_accuracies,
    'explanation_accuracy_inv': explanation_accuracies_inv,
    'elapsed_time': elapsed_times,
    'elapsed_time_inv': elapsed_times_inv,
})
results_lime.to_csv(os.path.join(results_dir, 'results_lime.csv'))
results_lime

,method,split,explanation,explanation_inv,model_accuracy,explanation_accuracy,explanation_accuracy_inv,elapsed_time,elapsed_time_inv
0,lime,0,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.0,1.000000,0.824880,0.769946
1,lime,1,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.0,1.000000,0.831771,0.893610
2,lime,2,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),0.888889,1.0,1.000000,0.791883,0.862787
3,lime,3,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.0,1.000000,0.878650,1.010077
4,lime,4,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.0,1.000000,0.832774,0.707105
5,lime,5,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.0,1.000000,0.745088,0.739088
6,lime,6,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.0,1.000000,0.717585,0.701133
7,lime,7,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.0,1.000000,0.847749,0.724077
8,lime,8,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.0,1.000000,0.699171,0.769941
9,lime,9,c1 | c2,~c1 & ~c2,0.888889,1.0,0.888889,0.512689,0.491686


# Weights

In [14]:
need_pruning = False
method = 'weights'
methods = []
splits = []
explanations = []
explanations_inv = []
model_accuracies = []
explanation_accuracies = []
explanation_accuracies_inv = []
elapsed_times = []
elapsed_times_inv = []
for seed in range(n_rep):
    print(f'Seed [{seed+1}/{n_rep}]')
    
    model = train_nn(x_train, y_train, need_pruning, seed, device)
    
    y_preds = model(x_test.to(device)).cpu().detach().numpy()
    model_accuracy = accuracy_score(y_test.cpu().detach().numpy(), y_preds.argmax(axis=1))
    print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
    
    # positive class
    target_class = 1
    start = time.time()
    global_explanation, _, _ = logic.relunn.combine_local_explanations(model, 
                                                                       x_train.to(device), y_train.to(device), 
                                                                       target_class=target_class,
                                                                       method=method, device=device)
    elapsed_time = time.time() - start
    if global_explanation:
        explanation_accuracy, _ = logic.base.test_explanation(global_explanation, target_class, x_test, y_test)
        explanation = logic.base.replace_names(global_explanation, concepts)
    print(f'\t Class {target_class} - Global explanation: "{explanation}" - Accuracy: {explanation_accuracy:.4f}')
    print(f'\t Elapsed time {elapsed_time}')
        
    # negative class
    target_class = 0
    start = time.time()
    global_explanation_inv, _, _ = logic.relunn.combine_local_explanations(model, 
                                                                           x_train.to(device), y_train.to(device), 
                                                                           target_class=target_class,
                                                                           method=method, device=device)
    elapsed_time_inv = time.time() - start
    if global_explanation_inv:
        explanation_accuracy_inv, _ = logic.base.test_explanation(global_explanation_inv, target_class, x_test, y_test)
        explanation_inv = logic.base.replace_names(global_explanation_inv, concepts)
    print(f'\t Class {target_class} - Global explanation: "{explanation_inv}" - Accuracy: {explanation_accuracy_inv:.4f}')
    print(f'\t Elapsed time {elapsed_time_inv}')
    
    methods.append(method)
    splits.append(seed)
    explanations.append(explanation)
    explanations_inv.append(explanation_inv)
    model_accuracies.append(model_accuracy)
    explanation_accuracies.append(explanation_accuracy)
    explanation_accuracies_inv.append(explanation_accuracy_inv)
    elapsed_times.append(elapsed_time)
    elapsed_times_inv.append(elapsed_time_inv)

Seed [1/10]
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 1.0000
	 Epoch 1000: train accuracy: 1.0000
	 Model's accuracy: 1.0000
	 Class 1 - Global explanation: "(c1 & ~c2) | (c2 & ~c1)" - Accuracy: 1.0000
	 Elapsed time 0.02091503143310547
	 Class 0 - Global explanation: "(c1 & c2) | (~c1 & ~c2)" - Accuracy: 1.0000
	 Elapsed time 0.020943164825439453
Seed [2/10]
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 1.0000
	 Epoch 1000: train accuracy: 1.0000
	 Model's accuracy: 1.0000
	 Class 1 - Global explanation: "(c1 & ~c2) | (c2 & ~c1)" - Accuracy: 1.0000
	 Elapsed time 0.02292656898498535
	 Class 0 - Global explanation: "(c1 & c2) | (~c1 & ~c2)" - Accuracy: 1.0000
	 Elapsed time 0.023935556411743164
Seed [3/10]
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 1.0000
	 Epoch 1000: train accuracy: 1.0000
	 Model's accuracy: 0.8889
	 Class 1 - Global explanation: "(c1 & ~c2) | (c2 & ~c1)" - Accuracy: 1.0000
	 Elapsed time 0.032911062240600586

In [16]:
results_weights = pd.DataFrame({
    'method': methods,
    'split': splits,
    'explanation': explanations,
    'explanation_inv': explanations_inv,
    'model_accuracy': model_accuracies,
    'explanation_accuracy': explanation_accuracies,
    'explanation_accuracy_inv': explanation_accuracies_inv,
    'elapsed_time': elapsed_times,
    'elapsed_time_inv': elapsed_times_inv,
})
results_weights.to_csv(os.path.join(results_dir, 'results_weights.csv'))
results_weights

,method,split,explanation,explanation_inv,model_accuracy,explanation_accuracy,explanation_accuracy_inv,elapsed_time,elapsed_time_inv
0,weights,0,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.0,1.000000,0.020915,0.020943
1,weights,1,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.0,1.000000,0.022927,0.023936
2,weights,2,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),0.888889,1.0,1.000000,0.032911,0.043901
3,weights,3,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.0,1.000000,0.025963,0.012930
4,weights,4,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.0,1.000000,0.020974,0.021911
5,weights,5,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.0,1.000000,0.019917,0.018984
6,weights,6,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.0,1.000000,0.019969,0.021008
7,weights,7,(c1 & ~c2) | (c2 & ~c1),c4 | (c1 & c2),1.000000,1.0,0.444444,0.030945,0.061844
8,weights,8,(c1 & ~c2) | (c2 & ~c1),c1 | ~c2,1.000000,1.0,0.777778,0.018949,0.017982
9,weights,9,(c1 & ~c2) | (c2 & ~c1),~c1 & ~c2,0.888889,1.0,0.888889,0.010045,0.014920


# Psi network

In [17]:
def train_psi_nn(x_train, y_train, need_pruning, seed, device):
    set_seed(seed)
    x_train = x_train.to(device)
    y_train = y_train.to(device).to(torch.float)
    layers = [
        torch.nn.Linear(x_train.size(1), 10),
        torch.nn.Sigmoid(),
        torch.nn.Linear(10, 4),
        torch.nn.Sigmoid(),
        torch.nn.Linear(4, 1),
        torch.nn.Sigmoid(),
    ]
    model = torch.nn.Sequential(*layers).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_form = torch.nn.BCELoss()
    model.train()
    for epoch in range(4001):
        # forward pass
        optimizer.zero_grad()
        y_pred = model(x_train).squeeze()
        # Compute Loss
        loss = loss_form(y_pred, y_train)

        for module in model.children():
            if isinstance(module, torch.nn.Linear):
                loss += 0.00001 * torch.norm(module.weight, 1)

        # backward pass
        loss.backward()
        optimizer.step()

        if epoch > 2000 and need_pruning:
            model = prune_equal_fanin(model, 2, validate=True, device=device)
            need_pruning = False
            
        # compute accuracy
        if epoch % 500 == 0:
            y_pred_d = y_pred > 0.5
            accuracy = y_pred_d.eq(y_train).sum().item() / y_train.size(0)
            print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
    
    return model

In [ ]:
need_pruning = True
method = 'psi'
methods = []
splits = []
explanations = []
explanations_inv = []
model_accuracies = []
explanation_accuracies = []
explanation_accuracies_inv = []
elapsed_times = []
elapsed_times_inv = []
for seed in range(n_rep):
    print(f'Seed [{seed+1}/{n_rep}]')
    
    # positive class
    target_class = 1
    model = train_psi_nn(x_train, y_train, need_pruning, seed, device)
    
    y_preds = model(x_test.to(device)).cpu().detach().numpy()
    model_accuracy = accuracy_score(y_test.cpu().detach().numpy(), y_preds > 0.5)
    print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
    
    start = time.time()
    global_explanation = logic.generate_fol_explanations(model, device)[0]
    elapsed_time = time.time() - start
    explanation_accuracy, _ = logic.base.test_explanation(global_explanation, target_class, x_test, y_test)
    explanation = logic.base.replace_names(global_explanation, concepts)
    print(f'\t Class {target_class} - Global explanation: "{explanation}" - Accuracy: {explanation_accuracy:.4f}')
    print(f'\t Elapsed time {elapsed_time}')
        
    # negative class
    target_class = 0
    model = train_psi_nn(x_train, y_train.eq(target_class), need_pruning, seed, device)
    
    y_preds = model(x_test.to(device)).cpu().detach().numpy()
    model_accuracy = accuracy_score(y_test.eq(target_class).cpu().detach().numpy(), y_preds > 0.5)
    print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
    
    start = time.time()
    global_explanation_inv = logic.generate_fol_explanations(model, device)[0]
    elapsed_time_inv = time.time() - start
    explanation_accuracy_inv, _ = logic.base.test_explanation(global_explanation_inv, 
                                                              target_class, x_test, y_test)
    explanation_inv = logic.base.replace_names(global_explanation_inv, concepts)
    print(f'\t Class {target_class} - Global explanation: "{explanation_inv}" - Accuracy: {explanation_accuracy_inv:.4f}')
    print(f'\t Elapsed time {elapsed_time_inv}')
    
    methods.append(method)
    splits.append(seed)
    explanations.append(explanation)
    explanations_inv.append(explanation_inv)
    model_accuracies.append(model_accuracy)
    explanation_accuracies.append(explanation_accuracy)
    explanation_accuracies_inv.append(explanation_accuracy_inv)
    elapsed_times.append(elapsed_time)
    elapsed_times_inv.append(elapsed_time_inv)

Seed [1/10]
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 0.7500
	 Epoch 1000: train accuracy: 0.7500
	 Epoch 1500: train accuracy: 0.7500
	 Epoch 2000: train accuracy: 1.0000
	 Epoch 2500: train accuracy: 0.5000
	 Epoch 3000: train accuracy: 1.0000
	 Epoch 3500: train accuracy: 1.0000
	 Epoch 4000: train accuracy: 1.0000
	 Model's accuracy: 1.0000
	 Class 1 - Global explanation: "((c1 & ~c2) | (c2 & ~c1))" - Accuracy: 1.0000
	 Elapsed time 0.038897037506103516
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 0.5000
	 Epoch 1000: train accuracy: 0.5000
	 Epoch 1500: train accuracy: 1.0000
	 Epoch 2000: train accuracy: 1.0000
	 Epoch 2500: train accuracy: 0.7500
	 Epoch 3000: train accuracy: 0.7500
	 Epoch 3500: train accuracy: 0.7500
	 Epoch 4000: train accuracy: 0.7500
	 Model's accuracy: 0.8889
	 Class 0 - Global explanation: "(~c1 & ~c2)" - Accuracy: 0.8889
	 Elapsed time 0.036900997161865234
Seed [2/10]
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: t

In [ ]:
results_psi = pd.DataFrame({
    'method': methods,
    'split': splits,
    'explanation': explanations,
    'explanation_inv': explanations_inv,
    'model_accuracy': model_accuracies,
    'explanation_accuracy': explanation_accuracies,
    'explanation_accuracy_inv': explanation_accuracies_inv,
    'elapsed_time': elapsed_times,
    'elapsed_time_inv': elapsed_times_inv,
})
results_psi.to_csv(os.path.join(results_dir, 'results_psi.csv'))
results_psi

# Decision tree

In [ ]:
need_pruning = False
method = 'decision_tree'
methods = []
splits = []
explanations = []
explanations_inv = []
model_accuracies = []
explanation_accuracies = []
explanation_accuracies_inv = []
elapsed_times = []
elapsed_times_inv = []
for seed in range(n_rep):
    print(f'Seed [{seed+1}/{n_rep}]')
    
    classifier = DecisionTreeClassifier(random_state=seed)
    classifier.fit(x_train.detach().numpy(), y_train.detach().numpy())
    y_preds = classifier.predict(x_test.detach().numpy())
    model_accuracy = accuracy_score(y_test.detach().numpy(), y_preds)
    print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
    
    target_class = 1
    start = time.time()
    explanation = tree_to_formula(classifier, concepts, target_class)
    elapsed_time = time.time() - start
    print(f'\t Class {target_class} - Global explanation: {explanation}')
    print(f'\t Elapsed time {elapsed_time}')
    
    target_class = 0
    start = time.time()
    explanation_inv = tree_to_formula(classifier, concepts, target_class)
    elapsed_time = time.time() - start
    print(f'\t Class {target_class} - Global explanation: {explanation_inv}')
    print(f'\t Elapsed time {elapsed_time_inv}')
    
    methods.append(method)
    splits.append(seed)
    explanations.append(explanation)
    explanations_inv.append(explanation_inv)
    model_accuracies.append(model_accuracy)
    explanation_accuracies.append(model_accuracy)
    explanation_accuracies_inv.append(model_accuracy)
    elapsed_times.append(0)
    elapsed_times_inv.append(0)

In [ ]:
results_tree = pd.DataFrame({
    'method': methods,
    'split': splits,
    'explanation': explanations,
    'explanation_inv': explanations_inv,
    'model_accuracy': model_accuracies,
    'explanation_accuracy': explanation_accuracies,
    'explanation_accuracy_inv': explanation_accuracies_inv,
    'elapsed_time': elapsed_times,
    'elapsed_time_inv': elapsed_times_inv,
})
results_tree.to_csv(os.path.join(results_dir, 'results_tree.csv'))
results_tree

# Summary

In [ ]:
cols = ['model_accuracy', 'explanation_accuracy', 'explanation_accuracy_inv', 'elapsed_time', 'elapsed_time_inv']
mean_cols = [f'{c}_mean' for c in cols]
sem_cols = [f'{c}_sem' for c in cols]

# pruning
df_mean = results_pruning[cols].mean()
df_sem = results_pruning[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_pruning = pd.concat([df_mean, df_sem])
summary_pruning.name = 'pruning'

# lime
df_mean = results_lime[cols].mean()
df_sem = results_lime[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_lime = pd.concat([df_mean, df_sem])
summary_lime.name = 'lime'

# weights
df_mean = results_weights[cols].mean()
df_sem = results_weights[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_weights = pd.concat([df_mean, df_sem])
summary_weights.name = 'weights'

# psi
df_mean = results_psi[cols].mean()
df_sem = results_psi[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_psi = pd.concat([df_mean, df_sem])
summary_psi.name = 'psi'

# tree
df_mean = results_tree[cols].mean()
df_sem = results_tree[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_tree = pd.concat([df_mean, df_sem])
summary_tree.name = 'tree'

summary = pd.concat([summary_pruning, summary_lime, summary_weights, summary_psi, summary_tree], axis=1).T
summary.columns = mean_cols + sem_cols
summary

In [ ]:
summary.to_csv(os.path.join(results_dir, 'summary.csv'))